In [ ]:
# setup cell
import sys
from pathlib import Path

# Add project root to Python path
project_root = Path.cwd().parent  # since notebooks/ is one level down
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

import typer
from stable_baselines3 import PPO

from uav_nav_obstacle_avoidance_rl import config
from uav_nav_obstacle_avoidance_rl.utils import env_helpers

logger = config.logger

In [ ]:
# import os
# os.system("Xvfb :1 -screen 0 1024x768x24 &")
# os.environ['DISPLAY'] = ':1'

In [ ]:
def record_videos(
    model,
    video_folder,
    num_episodes: int = 5,
    video_length: int = 1000,
    # Environment parameters
    num_targets: int = 1,
    flight_dome_size: float = 5.0,
    max_duration_seconds: float = 80.0,
    enable_obstacles: bool = True,
    visual_obstacles: bool = True,
    num_obstacles: tuple[int, int] = (0, 3),
    obstacle_types: list[str] = ["sphere", "box", "cylinder"],
    obstacle_size_range: tuple[float, float] = (0.1, 0.8),
    obstacle_min_distance_from_start: float = 1.0,
    obstacle_hight_range: tuple[float, float] = (0.1, 5.0),
):
    """
    record videos of a trained model
    """

    # environment config
    env_kwargs = {
        "num_targets": num_targets,
        "flight_dome_size": flight_dome_size,
        "max_duration_seconds": max_duration_seconds,
        "enable_obstacles": enable_obstacles,
        "visual_obstacles": visual_obstacles,
        "num_obstacles": num_obstacles,
        "obstacle_types": obstacle_types,
        "obstacle_size_range": obstacle_size_range,
        "obstacle_min_distance_from_start": obstacle_min_distance_from_start,
        "obstacle_hight_range": obstacle_hight_range,
        "render_mode": "rgb_array",
    }

    # record episodes
    for episode in range(num_episodes):
        logger.info(f"Recording episode {episode + 1}/{num_episodes}")
        
        # create a fresh environment for each episode
        video_env = env_helpers.make_voyager_for_recording(
            video_folder=video_folder,
            video_length=video_length,
            video_name=f"ep-{episode}",
            **env_kwargs,
        )

        obs = video_env.reset()
        done = False
        step_count = 0

        while not done and step_count < video_length:
            action, _ = model.predict(obs, deterministic=True)
            
            # VecEnv.step() returns (obs, rewards, dones, infos)
            obs, rewards, dones, infos = video_env.step(action)
            
            done = dones[0] if hasattr(dones, '__len__') else dones
            
            step_count += 1

        # close after each episode to save the video
        video_env.close()
        logger.info(f"Episode {episode + 1} completed in {step_count} steps")
    logger.info("Video recording completed")


In [ ]:
# display video in notebook
from pathlib import Path
import base64
from IPython.display import HTML, display

def show_videos(video_path, prefix):
    """
    Loop over all `video_path/{prefix}*.mp4`, embed each as an
    HTML5 <video> tag with a <figcaption> that shows the filename.
    """
    html_fragments = []
    for mp4 in Path(video_path).glob(f"{prefix}*.mp4"):
        # encode video
        video_b64 = base64.b64encode(mp4.read_bytes()).decode("ascii")
        # get just the filename (or use .stem for name without extension)
        caption = mp4.name
        # build a <figure> with video + caption
        fragment = f"""
        <figure style="margin-bottom: 1em;">
          <figcaption style="font-family: sans-serif; font-size: 0.9em; margin-bottom: 0.3em;">
            {caption}
          </figcaption>
          <video autoplay loop controls style="height: 400px;">
            <source src="data:video/mp4;base64,{video_b64}" type="video/mp4" />
            Sorry, your browser doesn’t support embedded videos.
          </video>
        </figure>
        """
        html_fragments.append(fragment)

    display(HTML("<br>".join(html_fragments)))

In [ ]:
# run env and record video
model_path = Path(
    "/Users/user/Documents/education/CAI/bachelor_thesis/uav-nav-obstacle-avoidance-rl/reports/wandb/run-20250827_151621-ozx3m8e3/files/models/best_model.zip"
)

parent_dir = model_path.parent.parent
video_path = parent_dir / "media" / "videos"
video_path.mkdir(parents=True, exist_ok=True)

model = PPO.load(model_path)

# run episodes and record video
record_videos(
    model,
    video_folder=video_path,
    num_episodes = 5,
    video_length = 1000,
)

In [ ]:
show_videos(video_path, prefix="ep")

In [ ]:
show_videos(
    '/Users/user/Documents/education/CAI/bachelor_thesis/uav-nav-obstacle-avoidance-rl/reports/examples',
    prefix="ep"
)